In [1]:
import pandas as pd
import numpy as np
import altair as alt
import ScraperFC as sfc
import traceback
from data_scraping import *
from pathlib import Path
import os
import requests
import json
from unidecode import unidecode
from datetime import datetime


c:\Users\edoar\anaconda3\envs\football\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
matches_file = os.path.join('data','Serie A','2023','Serie A_2023_matches_table.csv')
df = pd.read_csv(matches_file) 
team_csv_file = os.path.join('data','Serie A','2023', 'teams.csv')
all_teams_df = pd.read_csv(team_csv_file)

In [ ]:
import time


andamento_df = pd.DataFrame()
count = 0
df = df.loc[df['round']=='Regular season', :]
for team_id, team in all_teams_df.values:
    print(team_id, team)
    home_matches_df = df.loc[df.home_team==team,:].copy()
    
    home_matches_df['result'] = home_matches_df[['home_goal','away_goal']].apply(
        lambda score: 'V' if score.home_goal > score.away_goal\
            else ('N' if score.home_goal == score.away_goal else 'P'), axis=1
    )
    home_matches_df = home_matches_df[['gameweek','home_team','result']].rename(columns={'home_team':'team_name'})
    
    away_matches_df = df.loc[df.away_team==team,:].copy()
    
    away_matches_df['result'] = away_matches_df[['home_goal','away_goal']].apply(
        lambda score: 'V' if score.home_goal < score.away_goal\
            else ('N' if score.home_goal == score.away_goal else 'P'), axis=1
    )

    away_matches_df = away_matches_df[['gameweek','away_team','result']].rename(columns={'away_team':'team_name'})
    team_df = pd.concat((home_matches_df,away_matches_df)).sort_values(by='gameweek')
    team_df['team_id'] = team_id
    team_df['points'] = team_df.result.apply(
        lambda r: 3 if r == 'V' else (1 if r == 'N' else 0)
    ) 
    team_df['running_points'] = team_df.points.cumsum()
    
    andamento_df = pd.concat((andamento_df, team_df))
    count += 1
    #print(f'team:{team},iter:{count}, len: {len(andamento_df)}, len/count: {len(andamento_df)/count}')
    #print(team_df)
    #time.sleep(5)
    


    

In [34]:
andamento_df['Rank'] = andamento_df.groupby('gameweek')['running_points'].rank(method='first', ascending=False, na_option='bottom')

In [35]:
andamento_df

,gameweek,team_name,result,team_id,points,running_points,Rank
8,1.0,Napoli,V,d48ad4ff,3,3,1.0
15,2.0,Napoli,V,d48ad4ff,3,6,1.0
29,3.0,Napoli,N,d48ad4ff,1,7,1.0
37,4.0,Napoli,N,d48ad4ff,1,8,4.0
42,5.0,Napoli,V,d48ad4ff,3,11,2.0
...,...,...,...,...,...,...,...
337,34.0,Sampdoria,P,8ff9e3b3,0,17,20.0
349,35.0,Sampdoria,N,8ff9e3b3,1,18,20.0
353,36.0,Sampdoria,P,8ff9e3b3,0,18,20.0
360,37.0,Sampdoria,N,8ff9e3b3,1,19,20.0


In [2]:
from data_preprocessing import create_perfomarnce_trend
year=2023
league = 'Serie A'
create_perfomarnce_trend(year=year, league='Serie A', save=True)

,gameweek,team_name,result,team_id,points,running_points,Rank
8,1.0,Napoli,V,d48ad4ff,3,3,1.0
15,2.0,Napoli,V,d48ad4ff,3,6,1.0
29,3.0,Napoli,N,d48ad4ff,1,7,1.0
37,4.0,Napoli,N,d48ad4ff,1,8,4.0
42,5.0,Napoli,V,d48ad4ff,3,11,2.0
...,...,...,...,...,...,...,...
337,34.0,Sampdoria,P,8ff9e3b3,0,17,20.0
349,35.0,Sampdoria,N,8ff9e3b3,1,18,20.0
353,36.0,Sampdoria,P,8ff9e3b3,0,18,20.0
360,37.0,Sampdoria,N,8ff9e3b3,1,19,20.0


In [1]:
import altair as alt

source = alt.pd.DataFrame([
      {
        "question": "Question 1",
        "type": "Strongly disagree",
        "value": 24,
        "percentage": 0.7,
        "percentage_start": -19.1,
        "percentage_end": -18.4
      },
      {
        "question": "Question 1",
        "type": "Disagree",
        "value": 294,
        "percentage": 9.1,
        "percentage_start": -18.4,
        "percentage_end": -9.2
      },
      {
        "question": "Question 1",
        "type": "Neither agree nor disagree",
        "value": 594,
        "percentage": 18.5,
        "percentage_start": -9.2,
        "percentage_end": 9.2
      },
      {
        "question": "Question 1",
        "type": "Agree",
        "value": 1927,
        "percentage": 59.9,
        "percentage_start": 9.2,
        "percentage_end": 69.2
      },
      {
        "question": "Question 1",
        "type": "Strongly agree",
        "value": 376,
        "percentage": 11.7,
        "percentage_start": 69.2,
        "percentage_end": 80.9
      },

      {
        "question": "Question 2",
        "type": "Strongly disagree",
        "value": 2,
        "percentage": 18.2,
        "percentage_start": -36.4,
        "percentage_end": -18.2
      },
      {
        "question": "Question 2",
        "type": "Disagree",
        "value": 2,
        "percentage": 18.2,
        "percentage_start": -18.2,
        "percentage_end": 0
      },
      {
        "question": "Question 2",
        "type": "Neither agree nor disagree",
        "value": 0,
        "percentage": 0,
        "percentage_start": 0,
        "percentage_end": 0
      },
      {
        "question": "Question 2",
        "type": "Agree",
        "value": 7,
        "percentage": 63.6,
        "percentage_start": 0,
        "percentage_end": 63.6
      },
      {
        "question": "Question 2",
        "type": "Strongly agree",
        "value": 11,
        "percentage": 0,
        "percentage_start": 63.6,
        "percentage_end": 63.6
      },

      {
        "question": "Question 3",
        "type": "Strongly disagree",
        "value": 2,
        "percentage": 20,
        "percentage_start": -30,
        "percentage_end": -10
      },
      {
        "question": "Question 3",
        "type": "Disagree",
        "value": 0,
        "percentage": 0,
        "percentage_start": -10,
        "percentage_end": -10
      },
      {
        "question": "Question 3",
        "type": "Neither agree nor disagree",
        "value": 2,
        "percentage": 20,
        "percentage_start": -10,
        "percentage_end": 10
      },
      {
        "question": "Question 3",
        "type": "Agree",
        "value": 4,
        "percentage": 40,
        "percentage_start": 10,
        "percentage_end": 50
      },
      {
        "question": "Question 3",
        "type": "Strongly agree",
        "value": 2,
        "percentage": 20,
        "percentage_start": 50,
        "percentage_end": 70
      },

      {
        "question": "Question 4",
        "type": "Strongly disagree",
        "value": 0,
        "percentage": 0,
        "percentage_start": -15.6,
        "percentage_end": -15.6
      },
      {
        "question": "Question 4",
        "type": "Disagree",
        "value": 2,
        "percentage": 12.5,
        "percentage_start": -15.6,
        "percentage_end": -3.1
      },
      {
        "question": "Question 4",
        "type": "Neither agree nor disagree",
        "value": 1,
        "percentage": 6.3,
        "percentage_start": -3.1,
        "percentage_end": 3.1
      },
      {
        "question": "Question 4",
        "type": "Agree",
        "value": 7,
        "percentage": 43.8,
        "percentage_start": 3.1,
        "percentage_end": 46.9
      },
      {
        "question": "Question 4",
        "type": "Strongly agree",
        "value": 6,
        "percentage": 37.5,
        "percentage_start": 46.9,
        "percentage_end": 84.4
      },

      {
        "question": "Question 5",
        "type": "Strongly disagree",
        "value": 0,
        "percentage": 0,
        "percentage_start": -10.4,
        "percentage_end": -10.4
      },
      {
        "question": "Question 5",
        "type": "Disagree",
        "value": 1,
        "percentage": 4.2,
        "percentage_start": -10.4,
        "percentage_end": -6.3
      },
      {
        "question": "Question 5",
        "type": "Neither agree nor disagree",
        "value": 3,
        "percentage": 12.5,
        "percentage_start": -6.3,
        "percentage_end": 6.3
      },
      {
        "question": "Question 5",
        "type": "Agree",
        "value": 16,
        "percentage": 66.7,
        "percentage_start": 6.3,
        "percentage_end": 72.9
      },
      {
        "question": "Question 5",
        "type": "Strongly agree",
        "value": 4,
        "percentage": 16.7,
        "percentage_start": 72.9,
        "percentage_end": 89.6
      },

      {
        "question": "Question 6",
        "type": "Strongly disagree",
        "value": 1,
        "percentage": 6.3,
        "percentage_start": -18.8,
        "percentage_end": -12.5
      },
      {
        "question": "Question 6",
        "type": "Disagree",
        "value": 1,
        "percentage": 6.3,
        "percentage_start": -12.5,
        "percentage_end": -6.3
      },
      {
        "question": "Question 6",
        "type": "Neither agree nor disagree",
        "value": 2,
        "percentage": 12.5,
        "percentage_start": -6.3,
        "percentage_end": 6.3
      },
      {
        "question": "Question 6",
        "type": "Agree",
        "value": 9,
        "percentage": 56.3,
        "percentage_start": 6.3,
        "percentage_end": 62.5
      },
      {
        "question": "Question 6",
        "type": "Strongly agree",
        "value": 3,
        "percentage": 18.8,
        "percentage_start": 62.5,
        "percentage_end": 81.3
      },

      {
        "question": "Question 7",
        "type": "Strongly disagree",
        "value": 0,
        "percentage": 0,
        "percentage_start": -10,
        "percentage_end": -10
      },
      {
        "question": "Question 7",
        "type": "Disagree",
        "value": 0,
        "percentage": 0,
        "percentage_start": -10,
        "percentage_end": -10
      },
      {
        "question": "Question 7",
        "type": "Neither agree nor disagree",
        "value": 1,
        "percentage": 20,
        "percentage_start": -10,
        "percentage_end": 10
      },
      {
        "question": "Question 7",
        "type": "Agree",
        "value": 4,
        "percentage": 80,
        "percentage_start": 10,
        "percentage_end": 90
      },
      {
        "question": "Question 7",
        "type": "Strongly agree",
        "value": 0,
        "percentage": 0,
        "percentage_start": 90,
        "percentage_end": 90
      },

      {
        "question": "Question 8",
        "type": "Strongly disagree",
        "value": 0,
        "percentage": 0,
        "percentage_start": 0,
        "percentage_end": 0
      },
      {
        "question": "Question 8",
        "type": "Disagree",
        "value": 0,
        "percentage": 0,
        "percentage_start": 0,
        "percentage_end": 0
      },
      {
        "question": "Question 8",
        "type": "Neither agree nor disagree",
        "value": 0,
        "percentage": 0,
        "percentage_start": 0,
        "percentage_end": 0
      },
      {
        "question": "Question 8",
        "type": "Agree",
        "value": 0,
        "percentage": 0,
        "percentage_start": 0,
        "percentage_end": 0
      },
      {
        "question": "Question 8",
        "type": "Strongly agree",
        "value": 2,
        "percentage": 100,
        "percentage_start": 0,
        "percentage_end": 100
      }
])

color_scale = alt.Scale(
    domain=[
        "Strongly disagree",
        "Disagree",
        "Neither agree nor disagree",
        "Agree",
        "Strongly agree"
    ],
    range=["#c30d24", "#f3a583", "#cccccc", "#94c6da", "#1770ab"]
)

y_axis = alt.Axis(
    title='Question',
    offset=5,
    ticks=False,
    minExtent=60,
    domain=False
)

alt.Chart(source).mark_bar().encode(
    x='percentage_start:Q',
    x2='percentage_end:Q',
    y=alt.Y('question:N').axis(y_axis),
    color=alt.Color('type:N').title('Response').scale(color_scale),
)

alt.Chart(...)

In [ ]:
stats = alt.pd.DataFrame([
      {
        "question": "Question 1",
        "team": "Strongly disagree",
        "stats": "goal",
        "value": 24,
        "percentage": 0.7,
        "percentage_start": -19.1,
        "percentage_end": -18.4
      },
      {
        "question": "Question 1",
        "team": "Disagree",
        "stats": "goal",
        "value": 294,
        "percentage": 9.1,
        "percentage_start": -18.4,
        "percentage_end": -9.2
      },